In [ ]:
import sys
sys.executable

'c:\\Users\\carlo\\miniforge3\\envs\\dtu02452\\python.exe'

In [11]:
from pathlib import Path
import pandas as pd
import mne
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
import pyedflib


In [ ]:
data = Path("../data/Raw Data")
edf_files = sorted(list(DATA_DIR.glob("*.edf")) + list(DATA_DIR.glob("*.EDF")))
print("Found:", len(edf_files))
edf_files[:5]

edf_file = edf_files[0]
print("Using:", edf_file)

raw = mne.io.read_raw_edf(edf_file, preload=False, verbose=False)
raw
raw.load_data()

print("Duration (s):", raw.times[-1])
print("Sampling rate (Hz):", raw.info["sfreq"])
print("Channels:", raw.info["nchan"])
raw.ch_names[:30]

raw.plot(start=600, duration=20)